# Utils

> Base utilities for configuration management, environment wrappers, and evolutionary operators

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
import pickle
from pathlib import Path
from typing import Any, Dict, Union
import numpy as np

## Configuration Utilities

Functions for saving and loading configurations with JSON and pickle serialization.

In [ ]:
#| export
def save_config_json(config: Dict[str, Any], filepath: Union[str, Path]) -> None:
    """Save configuration dictionary to JSON file.
    
    Args:
        config: Configuration dictionary with pickleable values
        filepath: Path to save JSON file
    """
    filepath = Path(filepath)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    # Convert numpy arrays to lists for JSON serialization
    serializable_config = {}
    for key, value in config.items():
        if isinstance(value, np.ndarray):
            serializable_config[key] = value.tolist()
        elif isinstance(value, (set, frozenset)):
            serializable_config[key] = list(value)
        elif isinstance(value, dict):
            # Recursively handle nested dicts
            serializable_config[key] = {k: v.tolist() if isinstance(v, np.ndarray) else v 
                                        for k, v in value.items()}
        else:
            serializable_config[key] = value
    
    with open(filepath, 'w') as f:
        json.dump(serializable_config, f, indent=2)

In [ ]:
#| export
def load_config_json(filepath: Union[str, Path]) -> Dict[str, Any]:
    """Load configuration dictionary from JSON file.
    
    Args:
        filepath: Path to JSON configuration file
        
    Returns:
        Configuration dictionary
    """
    filepath = Path(filepath)
    with open(filepath, 'r') as f:
        config = json.load(f)
    
    # Convert lists back to numpy arrays where appropriate
    # This will be handled by the individual classes
    return config

In [ ]:
#| export
def save_config_pickle(config: Dict[str, Any], filepath: Union[str, Path]) -> None:
    """Save configuration dictionary to pickle file.
    
    Args:
        config: Configuration dictionary
        filepath: Path to save pickle file
    """
    filepath = Path(filepath)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    with open(filepath, 'wb') as f:
        pickle.dump(config, f)

In [ ]:
#| export
def load_config_pickle(filepath: Union[str, Path]) -> Dict[str, Any]:
    """Load configuration dictionary from pickle file.
    
    Args:
        filepath: Path to pickle configuration file
        
    Returns:
        Configuration dictionary
    """
    filepath = Path(filepath)
    with open(filepath, 'rb') as f:
        return pickle.load(f)

## Environment Wrapper Utilities

Helper functions for working with Gymnasium environments.

In [ ]:
#| export
import gymnasium as gym
from typing import Tuple, Optional

In [ ]:
#| export
def create_environment(env_name: str, seed: Optional[int] = None) -> gym.Env:
    """Create and initialize a Gymnasium environment.
    
    Args:
        env_name: Gymnasium environment identifier (e.g., 'CartPole-v1')
        seed: Random seed for reproducibility
        
    Returns:
        Initialized Gymnasium environment
    """
    env = gym.make(env_name)
    if seed is not None:
        env.reset(seed=seed)
    return env

In [ ]:
#| export
def get_env_properties(env: gym.Env) -> Dict[str, Any]:
    """Extract environment properties for configuration.
    
    Args:
        env: Gymnasium environment
        
    Returns:
        Dictionary with observation_space, action_space dimensions
    """
    obs_space = env.observation_space
    action_space = env.action_space
    
    properties = {
        'observation_dim': obs_space.shape[0] if hasattr(obs_space, 'shape') else 1,
        'action_dim': action_space.n if hasattr(action_space, 'n') else action_space.shape[0],
        'observation_space_type': type(obs_space).__name__,
        'action_space_type': type(action_space).__name__
    }
    
    return properties

## DEAP Toolbox Helper Functions

Utility functions for setting up DEAP evolutionary operators.

In [ ]:
#| export
from deap import base, creator, tools
from typing import Callable

In [ ]:
#| export
def setup_deap_fitness(minimize: bool = False) -> None:
    """Setup DEAP fitness class.
    
    Args:
        minimize: If True, fitness is minimized; if False, maximized
    """
    # Remove existing fitness class if it exists
    if hasattr(creator, "FitnessMax"):
        del creator.FitnessMax
    if hasattr(creator, "FitnessMin"):
        del creator.FitnessMin
    
    # Create fitness class
    if minimize:
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    else:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))

In [ ]:
#| export
def register_operators(toolbox: base.Toolbox,
                      individual_creator: Callable,
                      evaluator: Callable,
                      selector: Callable,
                      crossover: Callable,
                      mutator: Callable) -> None:
    """Register genetic operators in DEAP toolbox.
    
    Args:
        toolbox: DEAP toolbox to register operators in
        individual_creator: Function to create new individual
        evaluator: Fitness evaluation function
        selector: Selection operator (e.g., tools.selTournament)
        crossover: Crossover operator
        mutator: Mutation operator
    """
    toolbox.register("individual", individual_creator)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluator)
    toolbox.register("select", selector)
    toolbox.register("mate", crossover)
    toolbox.register("mutate", mutator)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()